In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import py_d3
%matplotlib inline
py_d3.load_ipython_extension(get_ipython())

In [2]:
# csvファイルの読み込み
row_data = pd.read_csv('tokyo2016.csv', index_col=0)

In [3]:
df = row_data.reset_index()
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
df['power'] = df['power'].values.astype(int)
daily_data = df.groupby('date')['power'].sum().reset_index()

In [4]:
daily_data[['date', 'power']].head()

,date,power
0,2016-04-01,71626
1,2016-04-02,69778
2,2016-04-03,63798
3,2016-04-04,70949
4,2016-04-05,76572


In [5]:
daily_data[['date', 'power']].to_csv('data.tsv', sep='\t')

In [12]:
%%d3

<style>

body {
  font: 10px sans-serif;
}

.axis path,
.axis line {
  fill: none;
  stroke: #000;
  shape-rendering: crispEdges;
}

.bar {
  fill: orange;
}

.bar:hover {
  fill: orangered ;
}

.x.axis path {
  display: none;
}

.d3-tip {
  line-height: 1;
  font-weight: bold;
  padding: 12px;
  background: rgba(0, 0, 0, 0.8);
  color: #fff;
  z-index: 10000;
  border-radius: 2px;
}

.d3-tip:after {
  box-sizing: border-box;
  display: inline;
  font-size: 10px;
  width: 100%;
  line-height: 1;
  color: rgba(0, 0, 0, 0.8);
  content: "\25BC";
  position: absolute;
  text-align: center;
}


.d3-tip.n:after {
  margin: -1px 0 0 0;
  top: 100%;
  left: 0;
}
</style>

<div></div>

<script src="http://labratrevenge.com/d3-tip/javascripts/d3.tip.v0.6.3.js"></script>
<script>

var margin = {top: 40, right: 20, bottom: 120, left: 70},
    width = 960 - margin.left - margin.right,
    height = 500 - margin.top - margin.bottom;

var formatPercent = d3.format(",.0f");

// var x = d3.scale.ordinal().rangeRoundBands([0, width], .1);
var x = d3.time.scale().range([0, width]);
var y = d3.scale.linear()
    .range([height, 0]);

var xAxis = d3.svg.axis()
    .scale(x)
    .orient("bottom");

var yAxis = d3.svg.axis()
    .scale(y)
    .orient("left")
    .tickFormat(formatPercent);

var svg = d3.select("div").append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

var tip = d3.tip()
.attr('class', 'd3-tip')
.offset([-50, 0])
.html(function(d) {
return "<div><strong>date:</strong> <span style='color:white'>" + d.date_str + "</span></div>" + "<div><strong>power:</strong> <span style='color:red'>" + d.power + "</span></div>";
})

d3.tsv("data.tsv", type, function(error, data) {
 data.forEach(function(d) {
     d.date_str = d.date;
     d.date = Date.parse(d.date);
  });
  x.domain([
    d3.min(data, function(d) { return d.date; }), // 開始日時
    d3.max(data, function(d) { return d.date; })  // 終了日時
  ])
  y.domain([0, d3.max(data, function(d) { return d.power; })]);
  svg.append("g")
      .attr("class", "x axis")
      .attr("transform", "translate(0," + height + ")")
      .call(xAxis);

  svg.append("g")
      .attr("class", "y axis")
      .call(yAxis)
    .append("text")
      .attr("transform", "rotate(-90)")
      .attr("y", 6)
      .attr("dy", ".71em")
      .style("text-anchor", "end")
      .text("Power");

  svg.selectAll(".bar")
      .data(data)
    .enter().append("rect")
      .attr("class", "bar")
      .attr("x", function(d) { return x(d.date); })
      .attr("width", '2')
      .attr("y", function(d) { return y(d.power); })
      .attr("height", function(d) { return height - y(d.power); })
      .on('mouseover', tip.show)
      .on('mouseout', tip.hide)

});

svg.call(tip);

function type(d) {
  d.power = +d.power;
  return d;
}

</script>